In [6]:
using Printf

# Enumerated types for available smoothers
@enum Smoother JACOBI GAUSS_SEIDEL SOR

# Simulation parameters
const NUM_NODES = 17        # 2^n +1 개
const MG_CYCLES = 40
const NUM_SWEEP = 3
const SMOOTHER = GAUSS_SEIDEL
const TOLERANCE = 12.0
const w_array = [2, 2, 2]
const NoA = length(w_array)

# Functions to allocate arrays for grid levels
function allocate_arrays(n_nodes)
    n_levels = 1
    nodes = n_nodes
    while (nodes - 1) % 2 == 0 && (nodes - 1) / 2 + 1 >= 3
        nodes = (nodes - 1) ÷ 2 + 1
        n_levels += 1
    end

    phi = Vector{Matrix{Float64}}(undef, n_levels)
    aux = Vector{Matrix{Float64}}(undef, n_levels)
    f = Vector{Matrix{Float64}}(undef, n_levels)
    x = Vector{Matrix{Float64}}(undef, n_levels)
    y = Vector{Matrix{Float64}}(undef, n_levels)
    phi_exact = Matrix{Float64}(undef, n_nodes, n_nodes)

    nodes = n_nodes
    for i_level in 1:n_levels
        phi[i_level] = zeros(nodes, nodes)
        aux[i_level] = zeros(nodes, nodes)
        f[i_level] = zeros(nodes, nodes)
        x[i_level] = zeros(nodes, nodes)
        y[i_level] = zeros(nodes, nodes)
        nodes = (nodes - 1) ÷ 2 + 1
    end
    return phi, phi_exact, f, x, y, aux, n_levels
end

# Function to initialize the mesh grid
function generate_fine_mesh!(x, y, n_nodes)
    for i in 1:n_nodes
        for j in 1:n_nodes
            x[i, j] = (i - 1) / (n_nodes - 1)
            y[i, j] = (j - 1) / (n_nodes - 1)
        end
    end
    println("Created fine cartesian grid ($n_nodes x $n_nodes)...")
end

# Coarsen the mesh recursively
function coarsen_mesh!(x, y, n_nodes, n_levels, level)
    if level == n_levels
        return
    else
        for i in 1:n_nodes
            for j in 1:n_nodes
                if i % 2 == 1 && j % 2 == 1
                    x[level + 1][(i + 1) ÷ 2, (j + 1) ÷ 2] = x[level][i, j]
                    y[level + 1][(i + 1) ÷ 2, (j + 1) ÷ 2] = y[level][i, j]
                end
            end
        end
        n_coarse = (n_nodes - 1) ÷ 2 + 1
        coarsen_mesh!(x, y, n_coarse, n_levels, level + 1)
    end
end

# Initialize solution and source term on fine mesh
function initialize_solution!(phi, phi_exact, f, x, y, n_nodes)
    for i in 1:n_nodes
        for j in 1:n_nodes
            if i == 1 || j == 1 || i == n_nodes || j == n_nodes
                phi[i, j] = exp(x[i, j]) * exp(-2.0 * y[i, j])
            else
                phi[i, j] = 0.0
            end
            phi_exact[i, j] = exp(x[i, j]) * exp(-2.0 * y[i, j])
            f[i, j] = -5.0 * exp(x[i, j]) * exp(-2.0 * y[i, j])
        end
    end
    println("Successfully initialized solution...")
end

# Smoothing functions
function smooth_gauss_seidel!(phi_level, f_level, n_nodes, n_sweeps)
    # Apply Gauss-Seidel smoothing to the specific level of phi and f.
    h2 = (1.0 / (n_nodes - 1))^2
    for _ in 1:n_sweeps
        for i in 2:n_nodes-1
            for j in 2:n_nodes-1
                phi_level[i, j] = (phi_level[i+1, j] + phi_level[i-1, j] + phi_level[i, j+1] + phi_level[i, j-1] + h2 * f_level[i, j]) / 4.0
            end
        end
    end
end

function smooth_total!(phi, f, aux, n_nodes, n_sweeps, level, smoother_type)
    # Apply smoothing to a specific level, based on the smoother type
    if smoother_type == JACOBI
        smooth_jacobi!(phi[level], f[level], aux[level], n_nodes, n_sweeps)
    elseif smoother_type == GAUSS_SEIDEL
        smooth_gauss_seidel!(phi[level], f[level], n_nodes, n_sweeps)
    else
        error("Unrecognized smoother type.")
    end
end

function multigrid_cycle!(phi, f, aux, n_nodes, n_sweeps, n_levels, flevel, w_array, smoother_type)
    base_level = n_levels
    level = 0
    direc = 0

    # Determine level and direction
    for i in 1:div(NoA, 2)
        if flevel < base_level - 1
            direc = 1
            level = n_levels - (base_level - flevel)
            break
        elseif NoA % 2 == 0 && flevel == base_level - 1 && i == div(NoA, 2)
            level = n_levels - (base_level - flevel)
            break
        elseif flevel < base_level + w_array[i] - 1
            direc = -1
            level = (n_levels - 2) - (flevel - base_level)
            break
        end
        base_level += 2 * w_array[i]
    end

    # Pre-smoothing
    smooth_total!(phi, f, aux, n_nodes, n_sweeps, level, smoother_type)

    # Recursive call based on direction
    if direc == 1
        # up_down! (implement this function for multigrid up-down recursion)
    elseif direc == -1
        # down_up! (implement this function for multigrid down-up recursion)
    end

    # Post-smoothing
    smooth_total!(phi, f, aux, n_nodes, n_sweeps, level, smoother_type)
end

# Error computation function
function compute_residual!(phi, f, aux, n_nodes)
    # Calculate the residual R(phi) = f - Laplacian(phi) at each node and the L2 norm
    h2 = (1.0 / (n_nodes - 1))^2
    norm = 0.0
    for i in 2:n_nodes-1
        for j in 2:n_nodes-1
            aux[i, j] = f[i, j] - (phi[i+1, j] + phi[i-1, j] + phi[i, j+1] + phi[i, j-1] - 4.0 * phi[i, j]) / h2
            norm += aux[i, j]^2
        end
    end
    return sqrt(norm)  # L2 norm of the residual
end

# Multigrid example with error reporting per cycle
function run_multigrid_example()
    n_nodes = NUM_NODES
    phi, phi_exact, f, x, y, aux, n_levels = allocate_arrays(n_nodes)
    
    # Fine mesh generation and initial solution setup
    generate_fine_mesh!(x[1], y[1], n_nodes)
    coarsen_mesh!(x, y, n_nodes, n_levels, 1)
    initialize_solution!(phi[1], phi_exact, f[1], x[1], y[1], n_nodes)

    # Initial residual calculation
    initial_residual = compute_residual!(phi[1], f[1], aux[1], n_nodes)
    println("         0     $(@sprintf("%.6e", initial_residual))   N/A")

    # Main multigrid cycles with error output
    for cycle in 1:MG_CYCLES
        multigrid_cycle!(phi, f, aux, n_nodes, NUM_SWEEP, n_levels, 1, w_array, SMOOTHER)
        current_residual = compute_residual!(phi[1], f[1], aux[1], n_nodes)
        l2_error = norm(phi[1] - phi_exact)  # Error relative to exact solution
        println("         $(cycle)     $(@sprintf("%.6e", current_residual))   $(@sprintf("%.6e", l2_error))")
        
        # Stop if tolerance is achieved
        if log10(initial_residual / current_residual) > TOLERANCE
            println("Converged after $cycle cycles with tolerance met.")
            break
        end
    end
end

# Run the example
run_multigrid_example()


Created fine cartesian grid (17 x 17)...
Successfully initialized solution...
         0     2.481271e+03   N/A
         1     3.841335e+02   7.503439e+00
         2     2.611567e+02   5.620225e+00
         3     2.150840e+02   4.362013e+00
         4     1.899361e+02   3.426260e+00
         5     1.737469e+02   2.704432e+00
         6     1.624419e+02   2.139179e+00
         7     1.541967e+02   1.693553e+00
         8     1.480349e+02   1.341174e+00
         9     1.433622e+02   1.062160e+00
         10     1.397846e+02   8.411176e-01
         11     1.370259e+02   6.659669e-01
         12     1.348872e+02   5.271733e-01
         13     1.332217e+02   4.171909e-01
         14     1.319202e+02   3.300412e-01
         15     1.309001e+02   2.609861e-01
         16     1.300988e+02   2.062701e-01
         17     1.294681e+02   1.629165e-01
         18     1.289711e+02   1.285664e-01
         19     1.285790e+02   1.013503e-01
         20     1.282692e+02   7.978687e-02
         21     1